In [128]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import scipy.stats as st

In [129]:
# Codes for each Data Frame

diabetes_csv = "Data/National_Data.csv"
diabetes_df = pd.read_csv(diabetes_csv)
diabetes_df

,yearid,fipscode,raceid,ageid,genderid,educationid,indicatorid,miscid,datasourceid,estimateid,SuppressID,datasetid,GeoLevel,Estimate,SeEstimate,LowerLimit,UpperLimit
0,1980,-1,0,5,0,0,1,0,30,61,0,40,National,1,0.1,0.9,1.2
1,1980,-1,0,5,0,0,1,0,30,101,0,40,National,915,76,764,1065
2,1980,-1,0,6,0,0,1,0,3,61,0,3,National,5.4,0.3,4.8,6
3,1980,-1,0,6,0,0,1,0,3,101,0,3,National,2337,143,2054,2621
4,1980,-1,0,6,0,0,1,0,30,61,0,40,National,5.4,0.3,4.8,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29097,2017,-1,1,100,0,0,2,0,3,72,0,31,National,6.1,0.5,5.2,7.1
29098,2017,-1,2,99,0,0,2,0,3,72,0,31,National,8.7,1.2,6.6,11.5
29099,2017,-1,2,100,0,0,2,0,3,72,0,31,National,9.2,1.3,7,12.1
29100,2017,-1,7,99,0,0,2,0,3,72,0,31,National,9.7,1.6,7,13.4


In [130]:
# Race DF

diabetes_df.columns

Index(['yearid', 'fipscode', 'raceid', 'ageid', 'genderid', 'educationid',
       'indicatorid', 'miscid', 'datasourceid', 'estimateid', 'SuppressID',
       'datasetid', 'GeoLevel', 'Estimate', 'SeEstimate', 'LowerLimit',
       'UpperLimit'],
      dtype='object')

In [131]:
diabetes_age_df = diabetes_df.loc [:,["indicatorid", "ageid", "estimateid", "datasetid", "raceid", "yearid","Estimate"]]
diabetes_age_df

,indicatorid,ageid,estimateid,datasetid,raceid,yearid,Estimate
0,1,5,61,40,0,1980,1
1,1,5,101,40,0,1980,915
2,1,6,61,3,0,1980,5.4
3,1,6,101,3,0,1980,2337
4,1,6,61,40,0,1980,5.4
...,...,...,...,...,...,...,...
29097,2,100,72,31,1,2017,6.1
29098,2,99,72,31,2,2017,8.7
29099,2,100,72,31,2,2017,9.2
29100,2,99,72,31,7,2017,9.7


In [132]:
# Race DF. Rename the raceid indicator using cut/bins.

# Create bins in which to place values based on races.
bins = [0,5,6,7,8]

# Create labels for these bins
labels = ["18-44", "45-64", "65-74", "75+"]

# Slice the data and place it into bins
diabetes_age_df["Age Range"] = pd.cut(diabetes_df["ageid"], bins, labels=labels)
diabetes_age_df = diabetes_age_df.dropna()
diabetes_age_df

,indicatorid,ageid,estimateid,datasetid,raceid,yearid,Estimate,Age Range
0,1,5,61,40,0,1980,1,18-44
1,1,5,101,40,0,1980,915,18-44
2,1,6,61,3,0,1980,5.4,45-64
3,1,6,101,3,0,1980,2337,45-64
4,1,6,61,40,0,1980,5.4,45-64
...,...,...,...,...,...,...,...,...
29059,2,5,72,31,0,2016,3.5,18-44
29060,2,6,72,31,0,2016,10.7,45-64
29080,2,4,72,31,0,2017,9.2,18-44
29081,2,5,72,31,0,2017,3.7,18-44


In [188]:
# Create a new df with the parameters by each variable / 1,000s (estimateid = 101)

age_analysis_pop = diabetes_age_df.loc[(diabetes_age_df["indicatorid"] == 1) & (diabetes_age_df["ageid"]== 5) & \
                  (diabetes_age_df["estimateid"] == 101) & (diabetes_age_df["datasetid"] == 40) & \
                  (diabetes_age_df["yearid"] >= 2000)]


age_analysis_pop = age_analysis_pop.reset_index(drop = True)
age_analysis_pop

,indicatorid,ageid,estimateid,datasetid,raceid,yearid,Estimate,Age Range
0,1,6,101,40,0,2000,5040,45-64
1,1,6,101,40,0,2001,5834,45-64
2,1,6,101,40,0,2002,6030,45-64
3,1,6,101,40,0,2003,6239,45-64
4,1,6,101,40,0,2004,6963,45-64
5,1,6,101,40,0,2005,7595,45-64
6,1,6,101,40,0,2006,7752,45-64
7,1,6,101,40,0,2007,8093,45-64
8,1,6,101,40,0,2008,9223,45-64
9,1,6,101,40,0,2009,9886,45-64


In [149]:
age_demo_df = age_analysis_pop.groupby("Age Range")
diabetes_count = age_demo_df['estimateid'].sum()
diabetes_count

purchase_avg = round(age_demo_df['yearid'].mean(),2)

purchasing_analysis_age = pd.DataFrame({"Total Count": diabetes_count, 
                                        "Percent of Players": purchase_avg, })

purchasing_analysis_age

,Total Count,Percent of Players
Age Range,,
18-44,0,NaN
45-64,3838,1998.5
65-74,3838,1998.5
75+,3838,1998.5


In [95]:
age_demo_df = age_demo_df.pivot_table(values=["Estimate"], index=["yearid"], columns=["Age"],aggfunc=np.sum)
age_demo_df

KeyError: 'Age'